# IR Lab Tutorial: Document Expansion with DocT5Query

TBD...

# Import All Libraries

In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client
ensure_pyterrier_is_loaded()
import pandas as pd
import pyterrier as pt
from tqdm import tqdm
import gzip
import json

tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


## DocT5 Query on a corpus with already high recall

TBD.

In [3]:
dataset = 'antique-test-20230107-training'
pt_dataset = pt.get_dataset(f'irds:ir-benchmarks/{dataset}')

In [4]:
bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)

In [5]:
pt_dataset.get_topics('text').head(3)

,qid,query
0,3990512,how can we get concentration onsomething
1,714612,why doesn t the water fall off earth if it s r...
2,2528767,how do i determine the charge of the iron ion ...


In [6]:
pt.Experiment(
    retr_systems=[bm25],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25'],
    eval_metrics=['recall_1000']
)

,name,recall_1000
0,BM25,0.788732


In [3]:
def doc_t5_query(dataset):
    docs = tira.get_run_output('ir-benchmarks/seanmacavaney/DocT5Query', dataset) + '/documents.jsonl.gz'
    with gzip.open(docs, 'rt') as f:
        for l in tqdm(f):
            l = json.loads(l)
            l['text'] = l['querygen']
            l['docno'] = l['doc_id']
            del l['doc_id']
            del l['querygen']
            yield l

def doc_t5_query_index(dataset):
    indexer = pt.IterDictIndexer("/tmp/index2", overwrite=True, meta={'docno': 100, 'text': 20480})
    index_ref = indexer.index(doc_t5_query(dataset))
    return pt.IndexFactory.of(index_ref)

#index = doc_t5_query_index(dataset)


In [7]:
index = pt.IndexFactory.of('/tmp/index2')

In [16]:
docs_retrieved_by_bm25 = {}

bm25_result = bm25(pt_dataset.get_topics('title'))

for _, i in tqdm(bm25_result.iterrows()):
    qid, docno = str(i['qid']), str(i['docno'])

    if qid not in docs_retrieved_by_bm25:
        docs_retrieved_by_bm25[qid] = set()
    
    docs_retrieved_by_bm25[qid].add(docno)


188633it [00:08, 21509.20it/s]


In [13]:
omit_already_retrieved_docs = lambda i: i[i.apply(lambda j: str(j['docno']) not in docs_retrieved_by_bm25[str(j['qid'])], axis=1)]
omit_already_retrieved_docs = pt.apply.generic(omit_already_retrieved_docs)

In [29]:
bm25_doct5query = pt.BatchRetrieve(index, wmodel="BM25")

bm25_doct5query_new = bm25_doct5query >> omit_already_retrieved_docs

In [30]:
pt.Experiment(
    retr_systems=[bm25_doct5query, bm25_doct5query_new],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['DocT5Query >> BM25', 'DocT5Query w.o. BM25 >> BM25'],
    eval_metrics=['recall_1000']
)

,name,recall_1000
0,DocT5Query >> BM25,0.534685
1,DocT5Query w.o. BM25 >> BM25,0.019399


## DocT5 Query on a corpus with lower recall

TBD

In [4]:
dataset = 'longeval-train-20230513-training'
pt_dataset = pt.get_dataset(f'irds:ir-benchmarks/{dataset}')

In [5]:
bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)

In [6]:
pt.Experiment(
    retr_systems=[bm25],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25'],
    eval_metrics=['recall_1000']
)

,name,recall_1000
0,BM25,0.496595


In [7]:
index = doc_t5_query_index(dataset)

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/doc-t5-query/2024-03-19-19-46-01.zip
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 60.8M/60.8M [00:01<00:00, 38.8MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/ir-benchmarks/longeval-train-20230513-training/seanmacavaney


16164it [00:07, 5062.00it/s]

11:34:18.676 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (doc062207101085) - further warnings are suppressed


1570734it [03:13, 8136.38it/s] 


11:37:49.947 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 110 empty documents


In [11]:
bm25_doct5query = pt.BatchRetrieve(index, wmodel="BM25") % 100

In [12]:
docs_retrieved_by_bm25 = {}

bm25_result = bm25(pt_dataset.get_topics('title'))

for _, i in tqdm(bm25_result.iterrows()):
    qid, docno = str(i['qid']), str(i['docno'])

    if qid not in docs_retrieved_by_bm25:
        docs_retrieved_by_bm25[qid] = set()
    
    docs_retrieved_by_bm25[qid].add(docno)


0it [00:00, ?it/s]

66567it [00:04, 13604.33it/s]


In [14]:
omit_already_retrieved_docs = lambda i: i[i.apply(lambda j: str(j['docno']) not in docs_retrieved_by_bm25[str(j['qid'])], axis=1)]
omit_already_retrieved_docs = pt.apply.generic(omit_already_retrieved_docs)

bm25_doct5query_new = bm25_doct5query >> omit_already_retrieved_docs

In [15]:
pt.Experiment(
    retr_systems=[bm25_doct5query, bm25_doct5query_new],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['DocT5Query >> BM25', 'DocT5Query w.o. BM25 >> BM25'],
    eval_metrics=['recall_1000']
)

,name,recall_1000
0,DocT5Query >> BM25,0.350889
1,DocT5Query w.o. BM25 >> BM25,0.066767
